In [1]:
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, IntVector, Formula
pandas2ri.activate()

%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# utils = importr('utils')
# utils.install_packages('ergm')

In [3]:
base = importr('base')
stats = importr('stats')
ergm = importr('ergm')
network = importr('network', on_conflict="warn",
                  robject_translations={
                      'as.tibble.network': 'as_tibble_network',
                      'as_tibble.network': 'as_tibble_network'
                  }
         )

/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "network" to Python symbols: 
-as_tibble_network -> as_tibble.network, as.tibble.network
  warn(msg)


### Load the simulated $A^t$

In [18]:
filename = '../data/res1.rds'
readRDS = robjects.r['readRDS']
res = readRDS(filename)
# res = np.array(df).astype(int)

In [19]:
resdic = dict(res.items())

In [20]:
df = resdic['nw']
df = np.array(df).astype(int)

In [21]:
resdic.get('desc')

'form: edge+mutual. diss: edge+mutual'


In [22]:
df.shape

(15, 10, 10)

## Initialize the params

In [27]:
num_nodes = df[0].shape[0]
# n = 20
T = len(df)
# num_changepts = 5

p_pos = 2 # edge + mutual
p_neg = 2 # edge + mutual
p = p_pos + p_neg

np.random.seed(112)
theta = np.random.normal(0, 3, size= (T , p)) 
z = np.copy(theta) 
u = np.zeros((T, p)) 


form_terms = ['edges', 'mutual']
diss_terms = ['edges', 'mutual']

In [28]:
init = {
    'theta': np.copy(theta),
    'z': z,
    'u': u
}

In [29]:
from mple_learn import stergmGraph
sim1 = stergmGraph(
    X = df,
    form_terms=form_terms,
    diss_terms=diss_terms,
    lam=[5],
    max_steps=100,
    newton_max_steps=100,
    admm_alpha=1,
    converge_tol=1e-5,
    verbose=1
)

In [30]:
theta_res, z, u, c = sim1.mple(initial_values=init, tau_inc=2, m=10)

[INFO] ADMM step #0
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9996916772371429
[INFO] dual_resnorm: 2.456711
[INFO] primal_resnorm: 1.603543
[INFO] convergence: 2.456711


[INFO] ADMM step #1
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9620300187180674
[INFO] dual_resnorm: 0.434358
[INFO] primal_resnorm: 0.385031
[INFO] convergence: 0.434358


[INFO] ADMM step #2
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9665576765604914
[INFO] dual_resnorm: 0.144236
[INFO] primal_resnorm: 0.474805
[INFO] convergence: 0.474805


[INFO] ADMM step #3
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.973150383987558
[INFO] dual_resnorm: 0.076557
[INFO] primal_resnorm: 0.624485
[INFO] convergence: 0.624485


[INFO] ADMM step #4
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9728767538122187
[INFO] dual_resnorm: 0.074516
[INFO] primal_resnorm: 0.507017
[INFO] convergence: 0.507017


[INFO]

	Updating z...
	Updating u...
[INFO] max mu : 
 0.9310002670993331
[INFO] dual_resnorm: 0.000742
[INFO] primal_resnorm: 0.000385
[INFO] convergence: 0.000742


[INFO] ADMM step #42
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9310024191924142
[INFO] dual_resnorm: 0.000662
[INFO] primal_resnorm: 0.000338
[INFO] convergence: 0.000662


[INFO] ADMM step #43
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9310041597327301
[INFO] dual_resnorm: 0.000591
[INFO] primal_resnorm: 0.000298
[INFO] convergence: 0.000591


[INFO] ADMM step #44
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9310055659599761
[INFO] dual_resnorm: 0.000527
[INFO] primal_resnorm: 0.000262
[INFO] convergence: 0.000527


[INFO] ADMM step #45
	Updating theta...
	Updating z...
	Updating u...
[INFO] max mu : 
 0.9310067009909703
[INFO] dual_resnorm: 0.000470
[INFO] primal_resnorm: 0.000231
[INFO] convergence: 0.000470


[INFO] ADMM step #46
	Updating theta...


In [35]:
c

9.319446900544869e-06

In [32]:
np.arange(14)[np.linalg.norm(np.diff(theta_res, axis=0), ord=2, axis=1) > 1]

array([4, 9])

In [33]:
np.linalg.norm(np.diff(theta_res, axis=0), ord=2, axis=1) > 0.8

array([False, False, False, False,  True, False, False, False, False,
        True, False, False, False, False])

In [34]:
np.round(theta_res,2)

array([[-1.39, -0.55,  1.58,  0.26],
       [-1.39, -0.55,  1.58,  0.26],
       [-1.39, -0.55,  1.58,  0.26],
       [-1.39, -0.55,  1.58,  0.26],
       [-1.39, -0.55,  1.58,  0.26],
       [ 0.39,  1.28,  1.19,  0.87],
       [ 0.39,  1.28,  1.19,  0.87],
       [ 0.39,  1.28,  1.19,  0.87],
       [ 0.39,  1.28,  1.19,  0.87],
       [ 0.39,  1.28,  1.19,  0.87],
       [ 0.87,  1.67, -0.38, -0.4 ],
       [ 0.91,  1.7 , -0.53, -0.45],
       [ 0.91,  1.7 , -0.53, -0.45],
       [ 0.91,  1.7 , -0.53, -0.45],
       [ 0.91,  1.7 , -0.53, -0.45]])

In [141]:
resdic['coefs_pos'][:,:3]
resdic['coefs_neg'][:,:3]

array([[-1.,  1.,  2.],
       [-2.,  2.,  1.]])

array([[ 2.07944154,  1.09861229, -0.69314718],
       [-2.        ,  2.        , -1.        ]])

In [258]:
np.append(resdic['coefs_pos'][:,:3], resdic['coefs_neg'][:,:3], axis=0).T

array([[-1.        , -2.        ,  2.07944154, -2.        ],
       [ 1.        ,  2.        ,  1.09861229,  2.        ],
       [ 2.        ,  1.        , -0.69314718, -1.        ]])

### Test Newton's method

In [9]:
hess_f = sim1.theta_update_hess_f(theta)
grad_f = sim1.theta_update_grad_f(theta, z, u)

In [39]:
theta_res = sim1.theta_update(init['theta'], z, u)

[INFO]	Inner Step # 0. Current diff is 1.0000001
[INFO] Loss = 33092.597021391324
max of mu is 0.9996489878379565
[INFO]	Inner Step # 1. Current diff is 9.617097546986221
[INFO] Loss = 11305.777727932968
max of mu is 0.9632169359769646
[INFO]	Inner Step # 2. Current diff is 0.2595090612892723
[INFO] Loss = 11861.002038092121
max of mu is 0.7239483473007992
[INFO]	Inner Step # 3. Current diff is 0.459800049101596
[INFO] Loss = 11586.37767534945
max of mu is 0.7829629395469192
[INFO]	Inner Step # 4. Current diff is 0.025880145267771548
[INFO] Loss = 11580.147453216698
max of mu is 0.788276237493941
[INFO]	Inner Step # 5. Current diff is 8.945294012274224e-05


In [140]:
theta_res

array([[-2.39290964,  0.60903965,  0.4352056 , -1.80261384],
       [-1.8178356 , -1.55117841,  2.27144208, -0.46653777],
       [-4.56820457, -3.919792  , -1.51145279,  2.52212431],
       [-4.73860946, -3.93336226,  1.89421578, -1.48659024],
       [-0.36294804,  0.09303787,  1.08081491,  1.98301898],
       [ 1.39896977,  1.27439511,  0.42027876,  1.07446576],
       [ 2.32866375,  2.83051687,  0.46795194,  2.46968428],
       [ 3.03979677,  0.28846171,  2.88408828,  0.77524795],
       [-1.99453383, -0.85852602,  2.52752802,  1.03164035],
       [-3.93030243,  1.13600796,  3.81299235,  1.31769738],
       [ 2.78059519,  0.91625975,  0.279195  , -0.67474795],
       [ 4.18951476,  2.79440973, -4.28815829, -3.36843649],
       [-0.167136  ,  1.79699814, -3.21230955,  0.16659357],
       [ 3.28416616,  1.59893526, -4.20601004,  2.47416368],
       [ 5.00130672, -0.09500702, -1.49619046, -2.11945833]])

### Test fused lasso

In [48]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(eng.genpath('../gfl/'), nargout=0)

### Test mple

In [ ]:
theta_res